In [74]:
def import_libraries():
    import pandas as pd
    import numpy as np
    from datetime import datetime
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size': 17})
    import folium
    import re
    from sklearn import preprocessing
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import KFold, train_test_split
    from sklearn.metrics import accuracy_score, precision_score, recall_score
    from sklearn.ensemble import RandomForestClassifier,AdaBoostRegressor
    from imblearn.over_sampling import SMOTE
    from sklearn.model_selection import GridSearchCV

In [75]:
pd.read_csv

<function pandas.io.parsers._make_parser_function.<locals>.parser_f(filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]], sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal: str = '.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)>